In [13]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import sys
import random
import animatplot as amp

In [14]:
K=1
M=1 
wind=0.3 # tomar valores entre 0.03 y 0.3
g=0.0098*2

In [15]:
def aceleracion(i,x,y):
    #rij=(x[i]-x[j])**2+(y[i]-y[j])**2
    #ax=-G*m[i]*(x[j]-x[i])/rij**(3/2)
    #ay=-G*m[i]*(y[j]-y[i])/rij**(3/2)
    if i==0:
        ax=0
        ay=0
    #esto es por si está empotrada
    #elif i==1:
        #ax=0
        #ay=0
    elif i==len(m)-1:
        ax=0
        ay=(K/M)*(y[i-1]-y[i]) - (1/M)*wind*(y[i]-y[i-1])/(np.sqrt((x[i]-x[i-1])**2 + (y[i]-y[i-1])**2)) - g
    else:
        #ax=(K/M)*(x[i+1]+x[i-1]-2*x[i]) + 0.01*(1/M)*wind*(x[i+1]-x[i])/(np.sqrt((x[i+1]-x[i])**2 + (y[i+1]-y[i])**2))
        #ay=(K/M)*(y[i+1]+y[i-1]-2*y[i]) - (1/M)*wind*(y[i+1]-y[i])/(np.sqrt((x[i+1]-x[i])**2 + (y[i+1]-y[i])**2))
        
        ax=(K/M)*(x[i+1]+x[i-1]-2*x[i]) + (1/M)*0.0001*(y[i+1]-y[i])/(np.sqrt((x[i+1]-x[i])**2 + (y[i+1]-y[i])**2))*wind*(y[i+1]-y[i])/(np.sqrt((x[i+1]-x[i])**2 + (y[i+1]-y[i])**2))
        ay=(K/M)*(y[i+1]+y[i-1]-2*y[i]) - (1/M)*wind*(y[i+1]-y[i])/(np.sqrt((x[i+1]-x[i])**2 + (y[i+1]-y[i])**2)) - g

    return ax,ay

In [16]:
def Fk(k,x,y,vx,vy):
    akx=0
    aky=0
    #for i in range(1,3):
        #if i!=k:
    aki=aceleracion(k,x,y)
    akx=aki[0]
    aky=aki[1]
    return np.array([vx[k],vy[k],akx,aky])

In [17]:
def siguiente_valor(x,y,vx,vy):
    k1=[]
    for k in range(len(m)):
        k1.append(h*Fk(k,x,y,vx,vy))

    x1=tuple(x[i]+k1[i][0]/2 for i in range(len(m)))
    y1=tuple(y[i]+k1[i][1]/2 for i in range(len(m)))
    vx1=tuple(vx[i]+k1[i][2]/2 for i in range(len(m)))
    vy1=tuple(vy[i]+k1[i][3]/2 for i in range(len(m)))

    k2=[]
    for k in range(len(m)):
        k2.append(h*Fk(k,x1,y1,vx1,vy1))

    x2=tuple(x[i]+k2[i][0]/2 for i in range(len(m)))
    y2=tuple(y[i]+k2[i][1]/2 for i in range(len(m)))
    vx2=tuple(vx[i]+k2[i][2]/2 for i in range(len(m)))
    vy2=tuple(vy[i]+k2[i][3]/2 for i in range(len(m)))

    k3=[]
    for k in range(len(m)):
        k3.append(h*Fk(k,x2,y2,vx2,vy2))

    x3=tuple(x[i]+k3[i][0] for i in range(len(m)))
    y3=tuple(y[i]+k3[i][1] for i in range(len(m)))
    vx3=tuple(vx[i]+k3[i][2] for i in range(len(m)))
    vy3=tuple(vy[i]+k3[i][3] for i in range(len(m)))

    k4=[]
    for k in range(len(m)):
        k4.append(h*Fk(k,x3,y3,vx3,vy3))

    xf=tuple(x[i]+(k1[i][0]+2*k2[i][0]+2*k3[i][0]+k4[i][0])/6 for i in range(len(m)))
    yf=tuple(y[i]+(k1[i][1]+2*k2[i][1]+2*k3[i][1]+k4[i][1])/6 for i in range(len(m)))
    vxf=tuple(vx[i]+(k1[i][2]+2*k2[i][2]+2*k3[i][2]+k4[i][2])/6 for i in range(len(m)))
    vyf=tuple(vy[i]+(k1[i][3]+2*k2[i][3]+2*k3[i][3]+k4[i][3])/6 for i in range(len(m)))

    return xf,yf,vxf,vyf

In [28]:
#Este es el melo
particles=20
y=[]
x=[]
for i in range(particles):
    y.append(random.random())
    x.append(i/particles)
y=np.sort(y)
vx = np.zeros(particles)
vy = np.zeros(particles)

m=np.zeros(particles)+1
N=60000
h=0.001
#G=6.67e-11

X=[x]
Y=[y]
Vx=[vx]
Vy=[vy]

In [ ]:
#Esto es para comparar con la analítica, solo cambia la posición incial, tener cuidado con los valores de aceleración del inicio

particles=20
y=[]
x=[]
kn=8
for i in range(particles):
    x.append(i/particles)
    y.append((np.sin(kn*x[i])+((np.sin(kn))/(np.sinh(kn)))*np.sinh(kn*x[i]))*4)
#y=np.sort(y)
vx = np.zeros(particles)
vy = np.zeros(particles)

m=np.zeros(particles)+1
N=40000
h=0.001
#G=6.67e-11

X=[x]
Y=[y]
Vx=[vx]
Vy=[vy]

In [29]:
for t in range(N):
    sig=siguiente_valor(X[t],Y[t],Vx[t],Vy[t])
    X.append(sig[0])
    Y.append(sig[1])
    Vx.append(sig[2])
    Vy.append(sig[3])

X=np.array(X)
Y=np.array(Y)

In [30]:
def actualizar(i):
    ax.clear()
    #tipo='nr'
    for t in range(len(m)):
        if tipo=='nr':
            x=X[:,t]
            y=Y[:,t]
            #tipo='r'
        else:
            x=X[:,t]-X[:,0]
            y=Y[:,t]-Y[:,0]
        #if t==0:
            #ax.plot(x[:i],y[:i],linewidth=0)
            #plt.plot(x[i],y[i],'o',markersize=5,color='y')
            #plt.plot(x[i],y[i],'--')
        #elif t==1:
            #ax.plot(x[:i],y[:i],linewidth=0)
            #plt.plot(x[i],y[i],'o',markersize=5,color='c')
            #plt.plot(x[i],y[i],'--')
        #else:
        ax.plot(21*x[:i],(1/3)*y[:i],linewidth=0)
        #plt.plot(x[i],y[i],'o',markersize=5,color='b')
    plt.plot(21*X[i,:],(1/3)*Y[i,:],'-o',markersize=5,color='b')
    ax.set_xlim(0,21)
    ax.set_ylim(-10,10)
    plt.xlabel('Horizontal position [cm]',fontsize=12)
    plt.ylabel('Vertical position [cm]',fontsize=12)
    plt.title('Flapping flag with high velocity air-flow',fontsize=15)
    plt.xticks(size = 12)
    plt.yticks(size = 12)
    plt.grid()

In [23]:
%matplotlib auto

Using matplotlib backend: QtAgg


In [31]:
tipo='nr'

fig=plt.figure()
ax=fig.gca()

ani=animation.FuncAnimation(fig,actualizar,range(0,N,300))
writergif = animation.PillowWriter(fps=30)
ani.save('Alejo_numerico.gif',writer=writergif)
plt.show()

In [ ]:
animation.FuncAnimation?

In [54]:
vvy=[]
for i in range(N):
    vvy.append(Vy[:][i][-1])

array([[ 0.02477714,  0.10317297,  0.11810997, ...,  0.82750986,
         0.83360833,  0.89026802],
       [ 0.02477714,  0.10317292,  0.11810995, ...,  0.82750983,
         0.83360832,  0.89026796],
       [ 0.02477714,  0.10317277,  0.1181099 , ...,  0.82750971,
         0.8336083 ,  0.89026777],
       ...,
       [ 0.02477714, -0.06583701, -0.22093779, ...,  0.41811241,
         0.45881172,  0.5169183 ],
       [ 0.02477714, -0.06581184, -0.22099167, ...,  0.41784095,
         0.45861714,  0.51672635],
       [ 0.02477714, -0.06578669, -0.22104541, ...,  0.41756929,
         0.45842251,  0.51653429]])